In [ ]:
import os
from urllib.request import urlretrieve
import zipfile

In [ ]:
# Dowload population data
data_url = "https://www.abs.gov.au/statistics/people/population/regional-population/2022-23/32180_ERP_2023_SA2_GDA2020.zip"
urlretrieve(data_url, "../data/landing/population.zip")

In [ ]:
# Extract the contents of the zip file
with zipfile.ZipFile("../data/landing/population.zip", 'r') as zip_ref:
    zip_ref.extractall("../data/population/")

In [ ]:
import geopandas as gpd

# Load the GeoPackage file
gdf = gpd.read_file("../data/population/32180_ERP_2023_SA2_GDA2020.gpkg")

# Check the first few rows of the dataset to understand its structure
print(gdf.head(1))

In [ ]:
gdf = gdf[gdf["State_name_2021"] == "Victoria"]
gdf["SA2_name_2021"].unique()

In [ ]:
gdf.columns

In [ ]:
# Calculate net migration
gdf["Net_migration_2021_22"] = gdf["Net_overseas_migration_2021_22"] + gdf["Net_internal_migration_2021_22"]
gdf["Net_migration_2022_23"] = gdf["Net_overseas_migration_2022_23"] + gdf["Net_internal_migration_2022_23"]

In [ ]:
# Calculate ERP/km2 from 2001-2023
for year in range(2001, 2024):
    gdf[f"ERP_per_km2_{year}"] = gdf[f"ERP_{year}"] / gdf["Area_km2"]

In [ ]:
gdf.columns

In [ ]:
import folium
import branca.colormap as cm

In [ ]:
# Create directories to store plots
os.makedirs("../plots/net_migrants/", exist_ok=True)
os.makedirs("../plots/erp/", exist_ok=True)
os.makedirs("../plots/area/", exist_ok=True)
os.makedirs("../plots/erp_per_km2/", exist_ok=True)

In [ ]:
def create_map(gdf, column, ouput_directory): 
    # Create a map centered on Victoria
    m = folium.Map(location=[-37.4713, 144.7852], zoom_start=7)

    # Create a color map using branca
    colormap = cm.LinearColormap(
        colors=['blue', 'white', 'yellow', 'orange', 'red'],
        vmin=gdf[column].min(),
        vmax=gdf[column].max(),
        caption=column
    )

     # Define a function to style the features
    def style_function(feature):
        return {
            'fillOpacity': 0.7,
            'weight': 0.5,
            'fillColor': colormap(feature['properties'][column]),
            'color': 'black'
        }

    # Add the GeoDataFrame to the map with Folium
    folium.GeoJson(
        gdf,
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(
            fields=['SA2_name_2021', column], 
            aliases=['Suburb Name', column]
        )
    ).add_to(m)

    # Add the colormap legend to the map
    colormap.add_to(m)

    # Save and display the map
    m.save(f"{ouput_directory}/suburbs_{column.lower()}.html")

In [ ]:
# Plot suburbs' net migrations from 2021-2023
for col in ['Net_internal_migration_2021_22', 'Net_overseas_migration_2021_22', 'Net_migration_2021_22', 
'Net_internal_migration_2022_23', 'Net_overseas_migration_2022_23', 'Net_migration_2022_23']:
    create_map(gdf, col, '../plots/net_migrants')

In [ ]:
# Plot suburbs' ERP from 2001-2023    
for col in ['ERP_2001', 'ERP_2002',
       'ERP_2003', 'ERP_2004', 'ERP_2005', 'ERP_2006', 'ERP_2007', 'ERP_2008',
       'ERP_2009', 'ERP_2010', 'ERP_2011', 'ERP_2012', 'ERP_2013', 'ERP_2014',
       'ERP_2015', 'ERP_2016', 'ERP_2017', 'ERP_2018', 'ERP_2019', 'ERP_2020',
       'ERP_2021', 'ERP_2022', 'ERP_2023']:
    create_map(gdf, col, '../plots/erp')

In [ ]:
# Plot suburbs' area
create_map(gdf,'Area_km2', '../plots/area')

In [ ]:
# Plot suburbs' ERP/km2 from 2001-2023
for col in ['ERP_per_km2_2001', 'ERP_per_km2_2002',
       'ERP_per_km2_2003', 'ERP_per_km2_2004', 'ERP_per_km2_2005',
       'ERP_per_km2_2006', 'ERP_per_km2_2007', 'ERP_per_km2_2008',
       'ERP_per_km2_2009', 'ERP_per_km2_2010', 'ERP_per_km2_2011',
       'ERP_per_km2_2012', 'ERP_per_km2_2013', 'ERP_per_km2_2014',
       'ERP_per_km2_2015', 'ERP_per_km2_2016', 'ERP_per_km2_2017',
       'ERP_per_km2_2018', 'ERP_per_km2_2019', 'ERP_per_km2_2020',
       'ERP_per_km2_2021', 'ERP_per_km2_2022', 'ERP_per_km2_2023']:
    create_map(gdf, col, '../plots/erp_per_km2')